In [5]:
import requests
import pandas as pd

In [6]:
zips = [90049, 60608, 60615]

In [7]:
API_URL="http://api.censusreporter.org/1.0/data/show/{release}?table_ids={table_ids}&geo_ids={geoids}"


In [8]:
def get_data(tables=None, geoids=None, release='latest'):
    if geoids is None:
        geoids = ['040|01000US']
    if tables is None:
        tables = ['B01001']

    url = API_URL.format(table_ids=','.join(tables).upper(), 
                         geoids=','.join(geoids), 
                         release=release)
    response = requests.get(url)
    return response.json()

In [9]:
def prep_for_pandas(json_data,include_moe=False):
    """Given a dict of dicts as they come from a Census Reporter API call, set it up to be amenable to pandas.DataFrame.from_dict"""
    result = {}
    for geoid, tables in json_data.items():
        flat = {}
        for table,values in tables.items():
            for kind, columns in values.items():
                if kind == 'estimate':
                    flat.update(columns)
                elif kind == 'error' and include_moe:
                    renamed = dict((k+"_moe",v) for k,v in columns.items())
                    flat.update(renamed)
        result[geoid] = flat
    return result


In [19]:
response = get_data(geoids = ["86000US" + str(z) for z in zips], tables = ['B17001'])
df = pd.DataFrame.from_dict(prep_for_pandas(response['data']),orient='index')

In [20]:
df

,B17001010,B17001049,B17001033,B17001053,B17001031,B17001018,B17001005,B17001020,B17001057,B17001041,...,B17001052,B17001035,B17001008,B17001024,B17001042,B17001014,B17001059,B17001055,B17001017,B17001037
86000US60608,1329,1662,1721,1857,50017,1123,230,1229,2658,3988,...,494,1845,102,1376,3085,802,1336,3076,10976,203
86000US60615,1776,653,966,1157,30199,341,54,378,2245,2008,...,198,925,24,948,1504,256,975,2214,5029,257
86000US90049,85,794,671,1060,32561,0,0,18,2353,2365,...,204,916,0,205,1878,102,1618,2509,1313,247
